In [91]:
# Импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Настройки Pandas
pd.set_option('display.max_columns', None)

In [92]:
# Чтение датасета
df = pd.read_csv('/Users/dinaragali/ds_elbrus/rent-real-estate/data/_data.csv')

In [93]:
# Удаляем столбец `Unnamed: 0`
df = df.iloc[:, 1:]

# Переименовываем столбцы
NEW_COLUMNS = {
    'ID  объявления':           'id',
    'Количество комнат':        'rooms',
    'Тип':                      'type',
    'Метро':                    'metro_station',
    'Адрес':                    'address',
    'Площадь, м2':              'area',
    'Дом':                      'building_type',
    'Парковка':                 'parking',
    'Цена':                     'price',
    'Телефоны':                 'phones',
    'Описание':                 'description',
    'Ремонт':                   'renovation',
    'Площадь комнат, м2':       'rooms_area',
    'Балкон':                   'balcony',
    'Окна':                     'windows',
    'Санузел':                  'bathroom',
    'Можно с детьми/животными': 'kids_pets',
    'Дополнительно':            'extra',
    'Название ЖК':              'complex_title',
    'Серия дома':               'building_series',
    'Высота потолков, м':       'ceiling_height',
    'Лифт':                     'elevator',
    'Мусоропровод':             'trash_chute',
    'Ссылка на объявление':     'listing_url',
}
df = df.rename(columns=NEW_COLUMNS)

### Колонка `rooms`

In [94]:
# Создаем столбец `num_rooms` с кол-ом комнат
df['num_rooms'] = df['rooms'].str.extract(r'(\d+)').astype(float)

### Колонка `metro_station`

In [95]:
# Выделим из колонки `metro_station` сведения о времени до метро
pattern = r"\((\d+)\s*мин"
minutes_str = df['metro_station'].str.extract(pattern, expand=False)
df['time_to_metro'] = pd.to_numeric(minutes_str, errors="coerce")

### Колонка `area`

In [96]:
# Добавляем столбец с общей площадью квартир
df['total area'] = df['area'].str.split('/').str[0].astype('float')

### Колонка `building_type`

In [97]:
# Добавляем столбец `floor` с номером этажа и `total_floors` с этажностью здания
df['floor'] = df['building_type'].str.extract(r'(\d{1,3})/').astype(float)
df['total_floors'] = df['building_type'].str.extract(r'/(\d{1,3})').astype(float)
df['parking'].unique()

array(['подземная', nan, 'наземная', 'открытая', 'многоуровневая',
       'на крыше'], dtype=object)

### Колонка `parking`

In [98]:
# категоризовываем колонку `parking` в колонку `parking_categories` от 0 до 5
df['parking'].fillna('нет', inplace=True)
df['parking_categories'] = pd.factorize(df['parking'])[0]

### Столбец `Price`

In [ ]:
# Создаем столбец с главной ценой
# r'([\d\.]+)' Это регулярное выражение, которое используется для поиска чисел с точкой. 
# \d — означает любую цифру от 0 до 9. 
# [\d\.]+ — это: «одна или больше цифр и/или точек подряд». 
df['main_price'] = df['price'].str.extract(r'([\d\.]+)').astype(float)

# Поработаем с залогом
# Ищем слово Залог, дефис, и число после него
# *-\s* это мы так страхуемся, чтобы захватились пробелы до и после дефиса"""
df['deposit'] = df['price'].str.extract(r'Залог\s*-\s*([\d\s]+)')[0]
df['deposit'] = df['deposit'].str.replace(' ', '').astype(float).fillna(0)

# Коммунальные услуги - выводим 1 если True и 0 если False
df['utilities_included'] = df['price'].str.contains('Коммунальные услуги включены', case=False, na=False).astype(int)


# Срок аренды
# Ищем слово Срок аренды, дефис, и слова после него
df['rental_term'] = df['price'].str.extract(r'Срок аренды\s*-\s*([а-яА-Яa-zA-Z ]+)')[0].str.strip().map({'Длительный': 1, 'На несколько месяцев': 0})

# Предоплата
# Ищем слово Предоплата, дефис, и слова после него
df['prepayment_months'] = df['price'].str.extract(r'Предоплата\s*(\d+)')
df['prepayment_months'] = df['prepayment_months'].astype(float).astype(float).fillna(0)


### Столбец `balcony`

In [100]:
# Добавляем столбец `presence_balcony` с информацией 0 - балконов/лоджий нет, 1 - балкон/лоджия есть
def extract_numbers(balcony):
    if isinstance(balcony, str): # проверяем на отсутвтвие Nun 
        return int(1)
    else:
        return int(0)
df['presence_balcony'] = df['balcony'].apply(extract_numbers)

### Столбец `bathroom`

In [101]:
# Добавляем столбец `numbers_toilets` с информацией о кол-ве санузлов
def numbers_toilet(bathroom):
    if isinstance(bathroom, str): #проверяем на отсутвтвие Nun 
        numbers = re.findall(r'\d+', bathroom)
        return sum(int(num) for num in numbers)
    else:
        return int(1)
df['numbers_toilets'] = df['bathroom'].apply(numbers_toilet)

### Столбец `kids_pets`

In [102]:
# Добавляем столбец `kids_allowed` с информацией допустима ли аренда с детьми
df['kids_allowed'] = np.where(df['kids_pets'].str.contains('детьми'), 1, 0)
# Добавляем столбец `pets_allowed` с информацией допустима ли аренда с животными
df['pets_allowed'] = np.where(df['kids_pets'].str.contains('животными'), 1, 0)

### Столбец `extra`

In [103]:
# добавляем столбец с наличием/отсутствием мебели
df["furniture"] = np.where(df["extra"].str.contains('Мебель в комнатах'), 1, 0)
# добавляем столбец с наличием/отсутствием холодильника
#df["fridge"] = np.where(df["extra"].isna(), np.nan, df["extra"].str.contains("Холодильник", na=False).astype(int))
df["fridge"] = np.where(df["extra"].str.contains('Холодильник'), 1, 0)
# добавляем столбец с наличием/отсутствием стиральной машины
# df["washing_machine"] = np.where(df["extra"].isna(), np.nan, df["extra"].str.contains("Стиральная машина", na=False).astype(int))
df["washing_machine"] = np.where(df["extra"].str.contains('Стиральная машина'), 1, 0)
df["condi"] = np.where(df["extra"].str.contains('Кондиционер'), 1, 0)
df["fur_in_kitch"] = np.where(df["extra"].str.contains('Мебель на кухне'), 1, 0)

In [104]:
df.head(5)

,id,rooms,type,metro_station,address,area,building_type,parking,price,phones,description,renovation,rooms_area,balcony,windows,bathroom,kids_pets,extra,complex_title,building_series,ceiling_height,elevator,trash_chute,listing_url,num_rooms,time_to_metro,total area,floor,total_floors,parking_categories,main_price,deposit,utilities_included,rental_term,prepayment_months,presence_balcony,numbers_toilets,kids_allowed,pets_allowed,furniture,fridge,washing_machine,condi,fur_in_kitch
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,4.0,9.0,200.0,5.0,16.0,0,500000.0,500000.0,1,1,1.0,0,1,1,1,1,1,1,1,1
1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,4.0,8.0,198.0,5.0,16.0,0,500000.0,500000.0,1,1,1.0,0,3,1,0,1,1,1,1,1
2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,4.0,7.0,200.0,5.0,16.0,0,500000.0,500000.0,0,1,1.0,0,3,1,0,1,1,1,1,1
3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,4.0,3.0,170.0,5.0,6.0,0,400000.0,400000.0,0,1,1.0,0,3,0,1,1,1,1,1,1
4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",нет,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615,2.0,7.0,58.0,12.0,26.0,1,225000.0,225000.0,1,1,1.0,0,2,1,1,1,1,1,0,1


### Столбец `ceiling_height`

In [105]:
# Заполняем пропущенные значения в столбце `ceiling_height` медианой
df.fillna({'ceiling_height': (df['ceiling_height'].median())}, inplace=True)

### Столбец `elevator`

In [106]:
# Вытащим из строк информацию о количестве лифтов
df['elevator_num'] = df['elevator'].str.extract(r'(\d{1,2})\).*(\d{1,2})\).*|(\d{1,2})\)').astype(float).sum(axis=1, min_count=1).fillna(0)

### Столбец `trash_chute`

In [107]:
# 0 - мусоропровода нет, 1 - мусоропровод имеется
df['trash_chute'] = df['trash_chute'].map({'Да': 1, 'Нет': 0}).astype(float)

In [108]:
df.columns

Index(['id', 'rooms', 'type', 'metro_station', 'address', 'area',
       'building_type', 'parking', 'price', 'phones', 'description',
       'renovation', 'rooms_area', 'balcony', 'windows', 'bathroom',
       'kids_pets', 'extra', 'complex_title', 'building_series',
       'ceiling_height', 'elevator', 'trash_chute', 'listing_url', 'num_rooms',
       'time_to_metro', 'total area', 'floor', 'total_floors',
       'parking_categories', 'main_price', 'deposit', 'utilities_included',
       'rental_term', 'prepayment_months', 'presence_balcony',
       'numbers_toilets', 'kids_allowed', 'pets_allowed', 'furniture',
       'fridge', 'washing_machine', 'condi', 'fur_in_kitch', 'elevator_num'],
      dtype='object')

In [109]:
df.head(5)

,id,rooms,type,metro_station,address,area,building_type,parking,price,phones,description,renovation,rooms_area,balcony,windows,bathroom,kids_pets,extra,complex_title,building_series,ceiling_height,elevator,trash_chute,listing_url,num_rooms,time_to_metro,total area,floor,total_floors,parking_categories,main_price,deposit,utilities_included,rental_term,prepayment_months,presence_balcony,numbers_toilets,kids_allowed,pets_allowed,furniture,fridge,washing_machine,condi,fur_in_kitch,elevator_num
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",1.0,https://www.cian.ru/rent/flat/271271157,4.0,9.0,200.0,5.0,16.0,0,500000.0,500000.0,1,1,1.0,0,1,1,1,1,1,1,1,1,5.0
1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",0.0,https://www.cian.ru/rent/flat/271634126,4.0,8.0,198.0,5.0,16.0,0,500000.0,500000.0,1,1,1.0,0,3,1,0,1,1,1,1,1,2.0
2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,4.0,7.0,200.0,5.0,16.0,0,500000.0,500000.0,0,1,1.0,0,3,1,0,1,1,1,1,1,1.0
3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,4.0,3.0,170.0,5.0,6.0,0,400000.0,400000.0,0,1,1.0,0,3,0,1,1,1,1,1,1,1.0
4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",нет,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",1.0,https://www.cian.ru/rent/flat/273614615,2.0,7.0,58.0,12.0,26.0,1,225000.0,225000.0,1,1,1.0,0,2,1,1,1,1,1,0,1,2.0


In [110]:
df['elevator_num'].unique()


array([ 5.,  2.,  1.,  0.,  3.,  4.,  6.,  8.,  7., 60., 16., 50.])

In [111]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23368 entries, 0 to 23367
Data columns (total 45 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  23368 non-null  int64  
 1   rooms               22327 non-null  object 
 2   type                23368 non-null  object 
 3   metro_station       22053 non-null  object 
 4   address             23368 non-null  object 
 5   area                23368 non-null  object 
 6   building_type       23368 non-null  object 
 7   parking             23368 non-null  object 
 8   price               23368 non-null  object 
 9   phones              23368 non-null  object 
 10  description         23368 non-null  object 
 11  renovation          20613 non-null  object 
 12  rooms_area          14458 non-null  object 
 13  balcony             15390 non-null  object 
 14  windows             16755 non-null  object 
 15  bathroom            20696 non-null  object 
 16  kids

In [112]:
df.head(5)

,id,rooms,type,metro_station,address,area,building_type,parking,price,phones,description,renovation,rooms_area,balcony,windows,bathroom,kids_pets,extra,complex_title,building_series,ceiling_height,elevator,trash_chute,listing_url,num_rooms,time_to_metro,total area,floor,total_floors,parking_categories,main_price,deposit,utilities_included,rental_term,prepayment_months,presence_balcony,numbers_toilets,kids_allowed,pets_allowed,furniture,fridge,washing_machine,condi,fur_in_kitch,elevator_num
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",1.0,https://www.cian.ru/rent/flat/271271157,4.0,9.0,200.0,5.0,16.0,0,500000.0,500000.0,1,1,1.0,0,1,1,1,1,1,1,1,1,5.0
1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",0.0,https://www.cian.ru/rent/flat/271634126,4.0,8.0,198.0,5.0,16.0,0,500000.0,500000.0,1,1,1.0,0,3,1,0,1,1,1,1,1,2.0
2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,4.0,7.0,200.0,5.0,16.0,0,500000.0,500000.0,0,1,1.0,0,3,1,0,1,1,1,1,1,1.0
3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,4.0,3.0,170.0,5.0,6.0,0,400000.0,400000.0,0,1,1.0,0,3,0,1,1,1,1,1,1,1.0
4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",нет,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",1.0,https://www.cian.ru/rent/flat/273614615,2.0,7.0,58.0,12.0,26.0,1,225000.0,225000.0,1,1,1.0,0,2,1,1,1,1,1,0,1,2.0


In [113]:
del_list = ['rooms_area', 'balcony', 'windows', 'bathroom', 'kids_pets', 'extra', 'complex_title', 'building_series', 'elevator','trash_chute']

In [114]:
df.drop(del_list, axis=1, inplace=True)
df.head(5)

,id,rooms,type,metro_station,address,area,building_type,parking,price,phones,description,renovation,ceiling_height,listing_url,num_rooms,time_to_metro,total area,floor,total_floors,parking_categories,main_price,deposit,utilities_included,rental_term,prepayment_months,presence_balcony,numbers_toilets,kids_allowed,pets_allowed,furniture,fridge,washing_machine,condi,fur_in_kitch,elevator_num
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,3.0,https://www.cian.ru/rent/flat/271271157,4.0,9.0,200.0,5.0,16.0,0,500000.0,500000.0,1,1,1.0,0,1,1,1,1,1,1,1,1,5.0
1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,3.5,https://www.cian.ru/rent/flat/271634126,4.0,8.0,198.0,5.0,16.0,0,500000.0,500000.0,1,1,1.0,0,3,1,0,1,1,1,1,1,2.0
2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,3.2,https://www.cian.ru/rent/flat/271173086,4.0,7.0,200.0,5.0,16.0,0,500000.0,500000.0,0,1,1.0,0,3,1,0,1,1,1,1,1,1.0
3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,3.2,https://www.cian.ru/rent/flat/272197456,4.0,3.0,170.0,5.0,6.0,0,400000.0,400000.0,0,1,1.0,0,3,0,1,1,1,1,1,1,1.0
4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",нет,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,3.9,https://www.cian.ru/rent/flat/273614615,2.0,7.0,58.0,12.0,26.0,1,225000.0,225000.0,1,1,1.0,0,2,1,1,1,1,1,0,1,2.0


In [115]:
df.dropna(inplace=True)

In [116]:
df.isna().sum()

id                    0
rooms                 0
type                  0
metro_station         0
address               0
area                  0
building_type         0
parking               0
price                 0
phones                0
description           0
renovation            0
ceiling_height        0
listing_url           0
num_rooms             0
time_to_metro         0
total area            0
floor                 0
total_floors          0
parking_categories    0
main_price            0
deposit               0
utilities_included    0
rental_term           0
prepayment_months     0
presence_balcony      0
numbers_toilets       0
kids_allowed          0
pets_allowed          0
furniture             0
fridge                0
washing_machine       0
condi                 0
fur_in_kitch          0
elevator_num          0
dtype: int64

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17637 entries, 0 to 23367
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  17637 non-null  int64  
 1   rooms               17637 non-null  object 
 2   type                17637 non-null  object 
 3   metro_station       17637 non-null  object 
 4   address             17637 non-null  object 
 5   area                17637 non-null  object 
 6   building_type       17637 non-null  object 
 7   parking             17637 non-null  object 
 8   price               17637 non-null  object 
 9   phones              17637 non-null  object 
 10  description         17637 non-null  object 
 11  renovation          17637 non-null  object 
 12  ceiling_height      17637 non-null  float64
 13  listing_url         17637 non-null  object 
 14  num_rooms           17637 non-null  float64
 15  time_to_metro       17637 non-null  float64
 16  total are

building_type
rooms
elevator
trash_cute
